In [25]:
"""
import eventlet
eventlet.monkey_patch()
print("eventlet.monkey_patch() called")
"""

import logging
from datetime import datetime, timedelta
from pathlib import Path
import typing

# Imports gevent, which does monkey patching. Import this first to avoid
# troubles with other imports.
from astra.simulator import flight, forecastEnvironment

import numpy as np
import requests

In [26]:
logging.basicConfig(level=logging.DEBUG)

np.random.seed(42)

days_to_launch = 1
launch_datetime = datetime.now() + timedelta(days=days_to_launch)


def make_launch_params(latitude: float, longitude: float):
    elevation_dataset = "FABDEM"
    elevation_url = (
        f"https://api.elevationapi.com/api/Elevation?lat={latitude}&lon={longitude}&dataSet={elevation_dataset}"
    )
    response = requests.get(elevation_url)
    print(response.text)
    elevation = response.json()["geoPoints"][0]["elevation"]
    return {
        "launchSiteLat": latitude,
        "launchSiteLon": longitude,
        "launchSiteElev": elevation,
        "launchTime": launch_datetime,
        "inflationTemperature": 10,
        "forceNonHD": False,
    }


def make_flight_params(
    balloon: str,
    nozzle_lift_kg: float,
    payload_train_weight_kg: float,
    parachute: str,
    number_of_sim_runs: int,
    train_equiv_sphere_diam: float,
    output_path: Path,
    debugging: bool = False,
):
    return {
        "balloonGasType": "gt96He",
        "balloonModel": balloon,
        "nozzleLift": nozzle_lift_kg,
        "payloadTrainWeight": payload_train_weight_kg,
        "parachuteModel": parachute,
        "numberOfSimRuns": number_of_sim_runs,
        "trainEquivSphereDiam": train_equiv_sphere_diam,
        "outputPath": output_path,
        "debugging": debugging,
        "log_to_file": True,
    }

In [27]:
import json
from pprint import pformat, pprint
import tempfile

launch_coords_possibilities = {
    "Kartanonrannan_koulu": (60.153144, 24.551671),
}
launch_coords = launch_coords_possibilities["Kartanonrannan_koulu"]
flight_train_mass_kg = 0.604
flight_train_equiv_sphere_diam = 0.285
sim_runs = 10
balloon = "SFB800"
nozzle_lift_kg = 1.6
parachute = "SFP800"

def run_sim(
     launch_params, flight_params, program_out_redirect_buffer: typing.TextIO | None = None, progress_handler=None,
):
    sim_environment = forecastEnvironment(**launch_params, progressHandler=progress_handler)
    the_flight = flight(
        **flight_params,
        environment=sim_environment,
        progress_stream=program_out_redirect_buffer
    )
    the_flight.run()


def find_likely_landing_area_polygon(astra_flight_json_filepath: Path):
    with open(astra_flight_json_filepath) as f:
        flight_data = json.load(f)
    landing_markers = flight_data["landingMarkers"]
    predicted_landing_sites = [Point(marker["lat"], marker["lon"]) for marker in landing_markers]


def main():
    output_dir = tempfile.TemporaryDirectory()
    output_path = Path(output_dir.name)
    output_filepath = output_path / "astra_out.py"
    launch_params_dict = make_launch_params(*launch_coords)
    flight_params_dict = make_flight_params(
        balloon=balloon,
        nozzle_lift_kg=nozzle_lift_kg,
        payload_train_weight_kg=flight_train_mass_kg,
        parachute=parachute,
        number_of_sim_runs=sim_runs,
        train_equiv_sphere_diam=flight_train_equiv_sphere_diam,
        output_path=output_filepath,
        debugging=True,
    )
    print(f"launch params: {pformat(launch_params_dict)}")
    print(f"flight params: {pformat(flight_params_dict)}")
    run_sim(launch_params_dict, flight_params_dict)




if __name__ == "__main__":
    main()


2023-05-23 11:35:20,166.166 DEBUG:urllib3.connectionpool Starting new HTTPS connection (1): api.elevationapi.com:443
2023-05-23 11:35:20,404.404 DEBUG:urllib3.connectionpool https://api.elevationapi.com:443 "GET /api/Elevation?lat=60.157502&lon=24.654051&dataSet=FABDEM HTTP/1.1" 200 895


{"message":"OK","geoPoints":[{"latitude":60.157502,"longitude":24.654051,"elevation":36.09736251831055}],"resultCount":1,"dataSet":{"name":"FABDEM","description":"Forest And Buildings removed Copernicus DEM","publicUrl":"https://data.bris.ac.uk/data/dataset/25wfy0f9ukoge2gs7a5mqpq2j7","resolutionMeters":30.0,"resolutionArcSeconds":1.0,"noDataValue":-9999,"fileFormat":{"name":"GeoTiff file","fileExtension":".tif","type":1,"registration":1},"attribution":{"subject":"Digital Elevation Model","text":"Laurence Hawker, Jeffrey Neal (2021): FABDEM","url":"https://data.bris.ac.uk/data/dataset/25wfy0f9ukoge2gs7a5mqpq2j7","acknowledgement":"Non-Commercial Government Licence for public sector information"},"dataSource":{"indexFilePath":"Data\\FABDEM","dataSourceType":2},"pointsPerDegree":3600,"srid":4326,"isListed":true,"allowMissingDataGeneration":true,"extentInfo":"Global"},"statusCode":200}
launch params: {'forceNonHD': False,
 'inflationTemperature': 10,
 'launchSiteElev': 36.09736251831055,


KeyboardInterrupt
2023-05-23T08:35:22Z


KeyboardInterrupt: 